In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import torch.optim as optim
from collections import defaultdict
import torch.distributions as distributions
import pandas as pd
from collections import Counter
import os
import torch.nn.functional as F
%matplotlib inline

## Example of RNN and LSTM using pytorch

Source: http://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html#sphx-glr-intermediate-char-rnn-generation-tutorial-py

In [2]:
input_directory = "/Users/tfolkman/projects/elder_folkman/content/post/"

In [3]:
all_lines = ""
for file in os.listdir(input_directory):
    if file == "Contact.md":
        continue
    with open(input_directory + file, "r") as f:
        meta_counter = 0
        for i, line in enumerate(f):
            if "+++" in line:
                meta_counter += 1
                continue
            elif meta_counter > 1:
                clean_line = line.strip().lower()
                if len(clean_line) == 0:
                    all_lines += "\n"
                else:
                    all_lines = all_lines + " " + clean_line

In [4]:
len(all_lines)

325408

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, embedding_dim, n_layers=2, bs=1):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.bs = bs
        self.word_embeddings = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=0.5)
        self.hidden2tag = nn.Linear(hidden_size, output_size)
        self.hidden = self.initHidden()

    def forward(self, input):
        embeds = self.word_embeddings(input)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(input), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(input), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

    def initHidden(self):
        return (torch.zeros(self.n_layers, self.bs, self.hidden_size),
                torch.zeros(self.n_layers, self.bs, self.hidden_size))

In [6]:
letters = set([c.lower() for word in all_lines for c in word])
letters_to_index = {l:i for i,l in enumerate(letters)}
index_to_letter = {i:l for l, i in letters_to_index.items()}

In [7]:
n_letters = len(letters)

In [8]:
sequence_length = 100
batch_size = 1

def inputTensor(line):
    return torch.LongTensor([letters_to_index[letter] for letter in line[:-1]])

def targetTensor(line):
    letter_indexes = [letters_to_index[line[li]] for li in range(1, len(line))]
    return torch.LongTensor(letter_indexes)

def randomTrainingPair():
    starting_position = np.random.randint(len(all_lines)-(sequence_length+1))
    return all_lines[starting_position:starting_position+(sequence_length+1)]

def randomTrainingExample():
    line = randomTrainingPair()
    input_line_tensor = Variable(inputTensor(line))
    target_line_tensor = Variable(targetTensor(line))
    return input_line_tensor, target_line_tensor

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
criterion = nn.NLLLoss()
lstm = LSTM(n_letters, 64, n_letters, 64)

optimizer = optim.Adam(lstm.parameters(), lr=.001)

def train(input_line_tensor, target_line_tensor):

    optimizer.zero_grad()

    output = lstm(input_line_tensor)
    loss = criterion(output, target_line_tensor)

    loss.backward(retain_graph=True)

    optimizer.step()

    return output, loss.data.item() / input_line_tensor.size()[0]

In [ ]:
n_iters = 20000
print_every = 100
plot_every = 100
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    input_t, target_t = randomTrainingExample()
    output, loss = train(input_t, target_t)
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

In [ ]:
plt.plot(all_losses)

In [ ]:
max_length = 5

# Sample from a category and starting letter
def sample(start_letter='\n'):
    output_name = ""
    input, _ = randomTrainingExample()

    for i in range(max_length):
        output = lstm(input)
        output = output.detach().cpu().numpy()
        topi = [np.random.choice(n_letters, p=np.exp(l)) for l in output]
        letters = "".join([index_to_letter[indx] for indx in topi])
        output_name += letters
        input = inputTensor(letters)

    return output_name